### 파일 삭제

In [ ]:
### 파일삭제 할 때

# 디렉토리 초기화
import os
import glob

# 경로 설정
train_image_dir = '/content/datasets/new_dataset/images/train'
val_image_dir = '/content/datasets/new_dataset/images/val'
train_label_dir = '/content/datasets/new_dataset/labels/train'
val_label_dir = '/content/datasets/new_dataset/labels/val'
train_mask_dir = '/content/datasets/new_dataset/masks/train'
val_mask_dir = '/content/datasets/new_dataset/masks/val'

# 디렉토리 목록
dirs_to_clear = [train_label_dir, val_label_dir, train_image_dir, val_image_dir,train_mask_dir, val_mask_dir]

# 파일 삭제
for dir_path in dirs_to_clear:
    files = glob.glob(os.path.join(dir_path, '*'))
    for f in files:
        os.remove(f)

### 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### import

In [ ]:
import os
import zipfile
import shutil

import xml.etree.ElementTree as ET

### 파일 경로 설정 및 압축 해제, yaml파일 생성

In [ ]:
# 여러 개의 ZIP 파일 경로
drive_paths = [
    '/content/drive/MyDrive/Bbox_17_new.zip',
    '/content/drive/MyDrive/Bbox_4_new.zip',
    '/content/drive/MyDrive/Bbox_2_new.zip',
]

# 로컬 디렉토리 설정
local_dir = '/content/Bbox_all'

# 로컬 디렉토리 생성
os.makedirs(local_dir, exist_ok=True)

# ZIP 파일을 하나의 디렉토리에 압축 해제
for drive_path in drive_paths:
    local_zip_path = os.path.join('/content', os.path.basename(drive_path))
    !cp "{drive_path}" "{local_zip_path}"
    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        zip_ref.extractall(local_dir)

# 압축 해제된 파일 확인
!ls -al "{local_dir}"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 3024
drwxr-xr-x 212 root root 12288 Jul  3 03:22 .
drwxr-xr-x   1 root root  4096 Jul  3 03:17 ..
drwxr-xr-x   2 root root 16384 Jul  3 03:21 Bbox_0151
drwxr-xr-x   2 root root  4096 Jul  3 03:21 Bbox_0152
drwxr-xr-x   2 root root  4096 Jul  3 03:21 Bbox_0153
drwxr-xr-x   2 root root  4096 Jul  3 03:21 Bbox_0154
drwxr-xr-x   2 root root 16384 Jul  3 03:21 Bbox_0155
drwxr-xr-x   2 root root 16384 Jul  3 03:21 Bbox_0156
drwxr-xr-x   2 root root 20480 Jul  3 03:21 Bbox_0157
drwxr-xr-x   2 root root 16384 Jul  3 03:21 Bbox_0158
drwxr-xr-x   2 root root 16384 Jul  3 03:21 Bbox_0159
drwxr-xr-x   2 root root  4096 Jul  3 03:21 Bbox_0160
drwxr-xr-x   2 root root  4096 Jul  3 03:21 Bbox_0161
drwxr-xr-x   2 root root 12288 Jul  3 03:21 Bbox_0162
drwxr-xr-x   2 root root 16384 Jul  3 03:21 Bbox_0163
drwxr-xr-x   2 root root 16384 Jul  3 03:21 Bbox_0164
drwxr-xr-x 

In [ ]:
# 베이스 디렉토리 설정
base_dir = '/content/datasets/new_dataset'

# 디렉토리 구조 정의
directories = [
    os.path.join(base_dir, 'images/train'),
    os.path.join(base_dir, 'images/val'),
    os.path.join(base_dir, 'images/test'),
    os.path.join(base_dir, 'labels/train'),
    os.path.join(base_dir, 'labels/val'),
    os.path.join(base_dir, 'labels/test')
]

# 디렉토리 생성
for directory in directories:
    os.makedirs(directory, exist_ok=True)

# 데이터셋 설정 파일 생성
dataset_yaml_content = """
train: images/train
val: images/val
test: images/test

names:
    0: 'wheelchair'
    1: 'stroller'
    2: 'stop'
    3: 'scooter'
    4: 'pole'
    5: 'movable_signage'
    6: 'motorcycle'
    7: 'fire_hydrant'
    8: 'dog'
    9: 'bollard'
    10: 'bicycle'
    11: 'bench'
    12: 'barricade'
"""

# dataset.yaml 파일 생성
with open(os.path.join(base_dir, 'dataset5.yaml'), 'w') as f:
    f.write(dataset_yaml_content)


### 이미지와 라벨 파일 YOLO 형식에 맞추어 생성
- 코랩 디스크 용량이 제한되어 있어 압축 풀어서 지정 디렉토리로 이동한 ZIP 파일은 삭제함

In [ ]:
# 라벨 목록
class_name = {
    0: 'wheelchair',
    1: 'stroller',
    2: 'stop',
    3: 'scooter',
    4: 'pole',
    5: 'movable_signage',
    6: 'motorcycle',
    7: 'fire_hydrant',
    8: 'dog',
    9: 'bollard',
    10: 'bicycle',
    11: 'bench',
    12: 'barricade',
}

# 압축 파일 해제
def unzip_file(zip_file_path, extract_dir):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"{zip_file_path} 압축 파일이 성공적으로 해제되었습니다.")

# YOLO 형식 변환 함수
def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

# XML 파일에서 정보를 추출하여 YOLO 형식의 텍스트 파일을 생성
def convert_annotation(image, output_label_path):
    size = (int(image.get('width')), int(image.get('height')))
    with open(output_label_path, 'w') as out_file:
        for obj in image.findall('box'):
            label = obj.get('label')
            if label is None or label not in class_name.values():
                continue
            cls_id = list(class_name.values()).index(label)
            xmlbox = (float(obj.get('xtl')), float(obj.get('xbr')), float(obj.get('ytl')), float(obj.get('ybr')))
            bb = convert(size, xmlbox)
            out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

# 이미지와 라벨 파일을 폴더로 이동시키는 함수
def process_images_and_labels(extract_dir, train_img_dir, train_label_dir):
    os.makedirs(train_img_dir, exist_ok=True)
    os.makedirs(train_label_dir, exist_ok=True)

    for folder_name in os.listdir(extract_dir):
        folder_path = os.path.join(extract_dir, folder_name)
        if not os.path.isdir(folder_path):
            continue

        for xml_file in os.listdir(folder_path):
            if xml_file.endswith('.xml'):
                xml_path = os.path.join(folder_path, xml_file)
                tree = ET.parse(xml_path)
                root = tree.getroot()

                for image in root.findall('image'):
                    image_name = image.get('name')
                    image_path = os.path.join(folder_path, image_name)

                    if os.path.exists(image_path):
                        output_label_path = os.path.join(train_label_dir, os.path.splitext(image_name)[0] + '.txt')
                        convert_annotation(image, output_label_path)
                        shutil.move(image_path, os.path.join(train_img_dir, image_name))

# Google Drive에서 여러 ZIP 파일 복사 및 압축 해제
def process_multiple_zips(drive_paths, extract_dir, train_img_dir, train_label_dir):
    for drive_path in drive_paths:
        local_zip_path = os.path.join('/content', os.path.basename(drive_path))
        !cp "{drive_path}" "{local_zip_path}"
        unzip_file(local_zip_path, extract_dir)

        # 이미지와 라벨 파일을 처리
        process_images_and_labels(extract_dir, train_img_dir, train_label_dir)

        # 압축 해제된 디렉토리 삭제
        shutil.rmtree(extract_dir)
        os.makedirs(extract_dir, exist_ok=True)

        # 로컬 ZIP 파일 삭제
        os.remove(local_zip_path)
        print(f"{local_zip_path} 삭제되었습니다.")

# 여러 개의 ZIP 파일 경로
drive_paths = [
    '/content/drive/MyDrive/Bbox_17_new.zip',
    '/content/drive/MyDrive/Bbox_4_new.zip',
    '/content/drive/MyDrive/Bbox_2_new.zip',
]

# 로컬 디렉토리 설정
extract_dir = '/content/Bbox_all'
train_img_dir = '/content/datasets/new_dataset/images/train'
train_label_dir = '/content/datasets/new_dataset/labels/train'

# Google Drive에서 ZIP 파일 복사 및 압축 해제
process_multiple_zips(drive_paths, extract_dir, train_img_dir, train_label_dir)

print("이미지와 라벨 파일이 성공적으로 처리되었습니다.")


/content/Bbox_17_new.zip 압축 파일이 성공적으로 해제되었습니다.
/content/Bbox_17_new.zip 삭제되었습니다.
/content/Bbox_4_new.zip 압축 파일이 성공적으로 해제되었습니다.
/content/Bbox_4_new.zip 삭제되었습니다.
/content/Bbox_2_new.zip 압축 파일이 성공적으로 해제되었습니다.
/content/Bbox_2_new.zip 삭제되었습니다.
이미지와 라벨 파일이 성공적으로 처리되었습니다.


### train데이터에서 valid데이터로 일부 파일 이동

In [ ]:
# 경로 설정
train_image_dir = '/content/datasets/new_dataset/images/train'
val_image_dir = '/content/datasets/new_dataset/images/val'
train_label_dir = '/content/datasets/new_dataset/labels/train'
val_label_dir = '/content/datasets/new_dataset/labels/val'

# 유효성 검사 폴더 생성
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

# 이미지 파일 이동
image_files = sorted(os.listdir(train_image_dir))
label_files = sorted(os.listdir(train_label_dir))

# 뒤에 70개 파일 선택
image_files_to_move = image_files[-70:]
label_files_to_move = label_files[-70:]

# 파일 이동
for file in image_files_to_move:
    shutil.move(os.path.join(train_image_dir, file), os.path.join(val_image_dir, file))

for file in label_files_to_move:
    shutil.move(os.path.join(train_label_dir, file), os.path.join(val_label_dir, file))

print("파일이 성공적으로 이동되었습니다.")


파일이 성공적으로 이동되었습니다.


### 모델링

#### 모델 정의

In [ ]:
#설치
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.7/793.7 kB 8.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from ultralytics import YOLO

In [ ]:
model2 = YOLO(model='yolov8n.pt', task='detect')

100%|██████████| 6.25M/6.25M [00:00<00:00, 127MB/s]


In [ ]:
# !pip install -U ultralytics

#### 모델 학습

In [ ]:
model2.train(data='/content/datasets/new_dataset/dataset5.yaml',
            epochs=150,
            patience=7,
            save=True,
            save_period=10,  # 10 에폭마다 모델 저장
            save_dir='/content/drive/MyDrive/빅프로젝트/모델/yolo_v8_1',  # 모델 저장 디렉터리
            # project='trained',
            # name='trained_model',
            exist_ok=False,
            pretrained=True,
            optimizer='auto',
            verbose=False,
            seed=77,
            resume=False,
            freeze=None
            )

Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/datasets/new_dataset/dataset5.yaml, epochs=150, time=None, patience=7, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=77, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

100%|██████████| 755k/755k [00:00<00:00, 19.6MB/s]


Overriding model.yaml nc=80 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


...

                   all         70        198       0.85      0.754      0.846      0.596
EarlyStopping: Training stopped early as no improvement observed in last 7 epochs. Best results observed at epoch 14, best model saved as best.pt.
To update EarlyStopping(patience=7) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



21 epochs completed in 2.464 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3008183 parameters, 0 gradients, 8.1 GFLOPs


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.17it/s]


                   all         70        198      0.848      0.718      0.854      0.615
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x795e985a7550>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,  

#### 모델 저장 및 성능 확인

In [ ]:
model2.save('/content/drive/MyDrive/빅프로젝트/모델/yolo_v8_1/yolov8n_150epoch.pt')

In [ ]:
model2.val()

Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3008183 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/datasets/new_dataset/labels/val.cache... 70 images, 1 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:04<00:00,  1.07it/s]


                   all         70        198      0.849      0.717      0.852      0.609
Speed: 0.1ms preprocess, 23.3ms inference, 0.0ms loss, 12.3ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x795e98311d20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,  

In [ ]:
from ultralytics import YOLO

loaded_model2= YOLO('/content/runs/detect/train/weights/best.pt')

#### 추가 이미지로 테스트

In [ ]:
results = loaded_model2.predict(source='/content/drive/MyDrive/test/12.png',
                        save=True,
                        line_width=3)


image 1/1 /content/drive/MyDrive/test/12.png: 384x640 1 pole, 1 dog, 5 bollards, 173.1ms
Speed: 4.3ms preprocess, 173.1ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict


In [ ]:
results = loaded_model2.predict(source='/content/drive/MyDrive/test/53.png',
                        save=True,
                        line_width=3)


image 1/1 /content/drive/MyDrive/test/53.png: 640x512 1 wheelchair, 1 bollard, 136.3ms
Speed: 4.8ms preprocess, 136.3ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 512)
Results saved to runs/detect/predict


In [ ]:
results = loaded_model2.predict(source='/content/drive/MyDrive/test/82.png',
                        save=True,
                        line_width=3)


image 1/1 /content/drive/MyDrive/test/82.png: 448x640 3 motorcycles, 135.4ms
Speed: 6.3ms preprocess, 135.4ms inference, 2.7ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict


In [ ]:
results = loaded_model2.predict(source='/content/drive/MyDrive/test/22.png',
                        save=True,
                        line_width=3)


image 1/1 /content/drive/MyDrive/test/22.png: 384x640 1 pole, 1 movable_signage, 5 bollards, 13.0ms
Speed: 4.2ms preprocess, 13.0ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict


In [ ]:
results = loaded_model2.predict(source='/content/drive/MyDrive/test/33.png',
                        save=True,
                        line_width=3)


image 1/1 /content/drive/MyDrive/test/33.png: 640x416 1 pole, 1 barricade, 138.9ms
Speed: 4.8ms preprocess, 138.9ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 416)
Results saved to runs/detect/predict


In [ ]:
results = loaded_model2.predict(source='/content/drive/MyDrive/test/44.png',
                        save=True,
                        line_width=3)


image 1/1 /content/drive/MyDrive/test/44.png: 480x640 1 pole, 2 bollards, 9.4ms
Speed: 3.1ms preprocess, 9.4ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict


In [ ]:
results = loaded_model2.predict(source='/content/drive/MyDrive/test/73.png',
                        save=True,
                        line_width=3)


image 1/1 /content/drive/MyDrive/test/73.png: 480x640 5 bollards, 13.6ms
Speed: 5.0ms preprocess, 13.6ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict
